# Practical classification of myPersonality.org users psychotypes with BERT

In this notebook I study a publicly available subset of myPersonality.org Facebook dataset. I contains almost 10 000 comments from 250 respondents, for whom we know their big-5 scores and some social graph analytics data (e.g. Network Size, Betweenness, Brokerage, Transitivity etc.).

In the first part of this notebook I analyze the coorelation between big-5 traits and social graph analytics data.

In the second part I am trying to predict their big-5 scores from their comments.

## References:
* https://github.com/sridharsrinivasan7/MyPersonality_NLP - annotations of extra fields (Betweenness, Brokerage, Transitivity etc.)


In [58]:
import math

import torch
import numpy as np
import pandas as pd


pd.set_option('display.max_rows', 1000)
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


print(torch.__version__)
# print(torch.backends.mps.is_available())

1.12.1


In [69]:
import pandas as pd


mypersonality = pd.read_csv("./data/mypersonality_final.csv", encoding='cp1252')

mypersonality.loc[mypersonality['cEXT'] == 'n', 'cEXT'] = 0
mypersonality.loc[mypersonality['cEXT'] == 'y', 'cEXT'] = 1

mypersonality.loc[mypersonality['cNEU'] == 'n', 'cNEU'] = 0
mypersonality.loc[mypersonality['cNEU'] == 'y', 'cNEU'] = 1

mypersonality.loc[mypersonality['cAGR'] == 'n', 'cAGR'] = 0
mypersonality.loc[mypersonality['cAGR'] == 'y', 'cAGR'] = 1

mypersonality.loc[mypersonality['cCON'] == 'n', 'cCON'] = 0
mypersonality.loc[mypersonality['cCON'] == 'y', 'cCON'] = 1

mypersonality.loc[mypersonality['cOPN'] == 'n', 'cOPN'] = 0
mypersonality.loc[mypersonality['cOPN'] == 'y', 'cOPN'] = 1

mypersonality.astype({'cEXT': 'int32', 'cNEU': 'int32', 'cAGR': 'int32', 'cCON': 'int32', 'cOPN': 'int32'}).dtypes

mypersonality

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.00,3.15,3.25,4.40,0,1,0,0,1,06/19/09 03:21 PM,180.0,14861.60,93.29,0.03,15661.0,0.49,0.10
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.00,3.15,3.25,4.40,0,1,0,0,1,07/02/09 08:41 AM,180.0,14861.60,93.29,0.03,15661.0,0.49,0.10
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.00,3.15,3.25,4.40,0,1,0,0,1,06/15/09 01:15 PM,180.0,14861.60,93.29,0.03,15661.0,0.49,0.10
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.00,3.15,3.25,4.40,0,1,0,0,1,06/22/09 04:48 AM,180.0,14861.60,93.29,0.03,15661.0,0.49,0.10
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.00,3.15,3.25,4.40,0,1,0,0,1,07/20/09 02:31 AM,180.0,14861.60,93.29,0.03,15661.0,0.49,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9912,deb899e426c1a5c66c24eeb0d7df6257,little things give you away.,2.15,2.15,4.10,2.90,4.60,0,0,1,0,1,01/28/10 11:48 AM,36.0,185.71,31.21,0.40,377.0,0.32,0.63
9913,ea28a927cb6663480ea33ca917c3c8ba,is wishing it was Saturday.,4.05,3.35,3.80,3.95,4.50,1,1,1,1,1,02/02/10 08:08 PM,83.0,2935.76,88.40,0.08,3120.0,0.47,0.26
9914,ea28a927cb6663480ea33ca917c3c8ba,is studying hard for the G.R.E.,4.05,3.35,3.80,3.95,4.50,1,1,1,1,1,02/09/10 08:04 PM,83.0,2935.76,88.40,0.08,3120.0,0.47,0.26
9915,5532642937eb3497a43e15dbb23a9d2d,snipers get more head,1.40,4.05,3.30,3.40,3.95,0,1,0,0,1,02/10/10 02:01 AM,154.0,11424.50,98.25,0.02,11510.0,0.49,0.05


In [70]:
people = mypersonality.drop_duplicates('#AUTHID')
people

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.00,3.15,3.25,4.40,0,1,0,0,1,06/19/09 03:21 PM,180.0,14861.60,93.29,0.03,15661.0,0.49,0.10
102,318bf822d4f2bd3920367560218619c0,has bed bugs..... ewwwww!,4.50,4.00,3.00,4.50,3.75,1,1,0,1,0,06/27/09 11:21 PM,318.0,49024.80,97.88,0.02,49584.0,0.49,0.06
171,ecbddbfe00e0f83cfdb802a7186061c7,"is stuck on Band-Aid brand, cuz Band-Aid's stu...",4.30,2.15,3.60,3.30,4.10,1,0,1,0,1,06/28/09 05:03 AM,739.0,267574.00,98.39,0.01,270029.0,0.50,0.07
236,db39f7b2aad360b1033ec1f8fcd57f9c,Back from vacation and tired,4.65,3.20,3.05,3.65,4.75,1,1,0,1,1,07/15/09 10:59 PM,102.0,4359.80,86.33,0.09,4668.0,0.46,0.36
246,4d035bd3fd8d9595d15cea9e388964be,had a great day at church...,3.70,2.90,3.40,3.35,4.05,1,1,0,0,1,07/19/09 05:40 PM,57.0,1509.50,98.02,0.05,1522.0,0.49,0.03
286,172400f46880b309ca5e97d322bb8f01,"I have no excuses, least of all for God. Like...",3.45,2.85,2.80,2.70,4.15,0,1,0,0,1,07/15/09 11:19 PM,122.0,6529.30,89.94,0.07,6893.0,0.47,0.26
359,200255966ca6e2636535b5b93ac04497,Four day camping trip!!! Holy Crap! this is go...,3.15,1.90,4.15,4.00,3.55,0,0,1,1,0,07/16/09 09:48 PM,75.0,2650.67,98.14,0.04,2660.0,0.49,0.05
396,5489ed38556af050d6a93e5d27b95dfb,"fie! Time, Inc. has blocked gchat... does this...",4.15,3.10,3.20,3.60,3.80,1,1,0,1,1,07/23/09 05:55 PM,950.0,438219.00,97.42,0.01,445454.0,0.50,0.09
406,b9a44bb356832160caccc9eb3614e0ce,is at work! >.<,3.65,3.15,3.45,2.35,4.50,1,1,0,0,1,07/16/09 02:19 PM,475.0,108866.00,97.11,0.02,110588.0,0.49,0.13
449,9a8cf1dcb2de1a8f70c4745a636bbb96,On her way to NOLA!,3.75,3.25,4.50,5.00,4.00,1,1,1,1,1,08/02/09 07:41 PM,251.0,30061.50,96.58,0.02,30616.0,0.49,0.11


## How well big-5 traits correlate with number of friends in Facebook (NETWORKSIZE)?

In [71]:
from scipy.stats import kendalltau


print(kendalltau(people['sEXT'], people['NETWORKSIZE']))
print(kendalltau(people['sNEU'], people['NETWORKSIZE']))
print(kendalltau(people['sAGR'], people['NETWORKSIZE']))
print(kendalltau(people['sCON'], people['NETWORKSIZE']))
print(kendalltau(people['sOPN'], people['NETWORKSIZE']))

KendalltauResult(correlation=0.272468146077793, pvalue=2.0261549342028602e-10)
KendalltauResult(correlation=-0.142213966455395, pvalue=0.0009138378003532861)
KendalltauResult(correlation=0.08636937105898886, pvalue=0.044414459578792614)
KendalltauResult(correlation=0.12456287908579083, pvalue=0.0037501161899842553)
KendalltauResult(correlation=0.011128400359359601, pvalue=0.7962111019510625)


## Try predicting the number of connections in Facebook from big-5 traits with a KRR

In [72]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split


n_samples, n_features = 10, 5
rng = np.random.RandomState(0)

y = people['NETWORKSIZE']
X = people[['sEXT', 'sNEU', 'sAGR', 'sCON', 'sOPN']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


krr = KernelRidge(alpha=1.0)
krr.fit(X_train, y_train)

KernelRidge(alpha=1.0)

In [73]:
predictions = krr.predict(X_test)
result = X_test.copy()
result['predictions'] = predictions
result['NETWORKSIZE'] = y_test
result

,sEXT,sNEU,sAGR,sCON,sOPN,predictions,NETWORKSIZE
7946,3.25,4.25,2.75,2.00,3.75,337.701113,169.0
359,3.15,1.90,4.15,4.00,3.55,446.903779,75.0
6358,3.20,3.60,3.85,4.35,4.80,385.530164,222.0
4289,3.70,3.15,4.30,3.75,3.85,485.361485,216.0
7159,3.10,2.00,3.25,3.90,4.60,377.774595,758.0
9249,4.05,1.35,3.85,3.50,4.80,510.212026,46.0
9471,2.45,2.95,3.30,3.55,4.60,281.133265,119.0
9288,1.90,2.70,4.10,4.20,4.45,260.529167,56.0
449,3.75,3.25,4.50,5.00,4.00,511.399757,251.0
6654,5.00,1.75,3.00,3.50,3.50,625.542553,855.0


## Training BERT classifier on Facebook comments

In [74]:
import torch
from torch.utils.data import DataLoader, random_split, default_convert
from transformers import AdamW, AutoTokenizer, BertForSequenceClassification
from datasets import Dataset, DatasetDict


model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, output_attentions=True)

# I'm running this on Apple Silicon. Activate Metal "mps" device, if available:
if torch.backends.mps.is_built():
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
        device = torch.device("cpu")
elif torch.cuda.is_available():
        device = torch.device("cuda:0")

else:
    print("Neither MPS, nor CUDA backends are available.")
    device = torch.device("cpu")
    
    
model.to(device)

model.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [83]:
# prepare dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(mypersonality):
    return tokenizer(mypersonality["STATUS"], padding="max_length", truncation=True)  # , return_tensors="pt")

mypersonality_dataset = Dataset.from_pandas(mypersonality)
tokenized_dataset = mypersonality_dataset.map(tokenize_function, batched=True, batch_size=8)
tokenized_dataset = tokenized_dataset.rename_column("STATUS", "text")
tokenized_dataset = tokenized_dataset.rename_column("cNEU", "labels")
tokenized_dataset = tokenized_dataset.remove_columns(["#AUTHID", "sEXT", "sNEU", "sAGR", "sCON", "sOPN", "cEXT", "cAGR", "cCON", "cOPN", "DATE", "NETWORKSIZE", "BETWEENNESS", "NBETWEENNESS", "DENSITY", "BROKERAGE", "NBROKERAGE", "TRANSITIVITY"])

train_dataset, validation_dataset = random_split(tokenized_dataset, [8000, len(tokenized_dataset) - 8000])

ds = DatasetDict()
ds['train'] = train_dataset
ds['validation'] = validation_dataset

# vocab = tokenizer.get_vocab()
# ivocab = {v: k for k, v in vocab.items()}
print(ds['train'][0]['input_ids'])

train_dataloader = DataLoader(ds['train'], shuffle=True, batch_size=8)

100%|███████████████████████████████████████████████| 1240/1240 [00:02<00:00, 489.02ba/s]


[101, 3407, 15060, 2000, 2035, 2026, 9130, 2814, 1998, 2155, 1012, 1045, 2572, 18836, 2005, 2035, 1997, 2017, 999, 2031, 1037, 6919, 2154, 2007, 6737, 999, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [92]:
from transformers import get_scheduler
from tqdm.auto import tqdm
from torch.optim import AdamW


# parameters
num_epochs = 1  # 3
num_training_steps = num_epochs * len(train_dataloader)

cross_entropy_loss = torch.nn.CrossEntropyLoss().to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)


# # test on one batch
# batch = next(iter(train_dataloader))

# labels = batch["labels"]
# del batch["labels"]
# del batch["text"]

# batch = {k: torch.transpose(torch.stack(default_convert(v)), 0, 1) for k, v in batch.items()}
# batch = {k: v.to(device) for k, v in batch.items()}

# output = model(**batch)
# labels.to(device)
# device_labels = torch.as_tensor(labels, device=device)

# loss = cross_entropy_loss(output.logits, device_labels)
# loss.backward()


# progress bar
progress_bar = tqdm(range(num_training_steps))

# training
for epoch in range(num_epochs):
    for batch in train_dataloader:
        labels = batch["labels"]
        del batch["labels"]
        del batch["text"]

        batch = {k: torch.transpose(torch.stack(default_convert(v)), 0, 1) for k, v in batch.items()}
        batch = {k: v.to(device) for k, v in batch.items()}

        output = model(**batch)
        labels.to(device)
        device_labels = torch.as_tensor(labels, device=device)

        loss = cross_entropy_loss(output.logits, device_labels)

        # loss = cross_entropy_loss(output.logits, mps_labels)
        print("Training loss: {0:.2f}".format(loss))
        loss.backward()        

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|                                                        | 0/1000 [16:30:46<?, ?it/s]


Training loss: 0.68


  0%|                                                 | 1/1000 [00:06<1:43:37,  6.22s/it]

Training loss: 0.78


  0%|                                                 | 2/1000 [00:19<2:55:17, 10.54s/it]

...

  4%|█▋                                              | 36/1000 [09:10<4:22:39, 16.35s/it]

Training loss: 0.52


  4%|█▊                                              | 37/1000 [09:29<4:32:50, 17.00s/it]

Training loss: 0.67


KeyboardInterrupt: 

In [95]:
from datasets import load_metric


validation_dataloader = DataLoader(ds['validation'], shuffle=True, batch_size=8)

metric = load_metric("accuracy")
model.eval()
for batch in validation_dataloader:
    labels = batch["labels"]
    device_labels = torch.as_tensor(labels, device=device)
    del batch["labels"]
    del batch["text"]

    batch = {k: torch.transpose(torch.stack(default_convert(v)), 0, 1) for k, v in batch.items()}
    batch = {k: v.to(device) for k, v in batch.items()}
    
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    softmax = torch.nn.Softmax()
    # for index, item in enumerate(logits):
        # print(f"probabilities = {softmax(item)}")
        # print(f"label = {labels[index]}")
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=device_labels)

metric.compute()

{'accuracy': 0.631716223265519}